In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../res/prepared_data.csv')

In [3]:
from sklearn.ensemble import RandomForestRegressor  # або RandomForestClassifier для класифікації
from sklearn.model_selection import train_test_split

# 1. Виберіть ознаки (X) та ціль (y)
X = df.drop(['home_club_goals', 'away_club_goals'], axis=1)
y = df[['home_club_goals', 'away_club_goals']]  # для регресії двох цілей

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 2. Створіть і навчіть модель
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 3. Передбачення
y_pred = model.predict(X_test)

In [4]:
import joblib

scaler = joblib.load('../res/scaler_y.joblib')

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

y_test_unscale = scaler.inverse_transform(y_test)
y_pred_unscale = scaler.inverse_transform(y_pred)

# MSE для кожної цілі
mse = mean_squared_error(y_test_unscale, y_pred_unscale)
print("MSE:", mse)

# L1 (MAE)
mae = mean_absolute_error(y_test_unscale, y_pred_unscale)
print("MAE (L1):", mae)

# R^2
r2 = r2_score(y_test_unscale, y_pred_unscale)
print("R^2:", r2)

# "Accuracy" — обидва передбачення в межах 0.5 від істинного значення
diff = np.abs(y_pred_unscale - y_test_unscale)
correct = np.all(diff <= 0.5, axis=1)  # True якщо і home, і away в межах 0.5
accuracy = np.mean(correct)
print("Custom accuracy (обидва <= 0.5):", accuracy)

# Функція для визначення класу результату
def get_result_class(goals):
    # goals: масив [n_samples, 2] (home, away)
    diff = goals[:, 0] - goals[:, 1]
    res = np.zeros_like(diff)
    res[diff > 0] = 1    # перемога домашньої
    res[diff < 0] = -1   # поразка домашньої
    # нічия залишиться 0
    return res

# Класи для справжніх і передбачених результатів
true_result = get_result_class(y_test_unscale)
pred_result = get_result_class(y_pred_unscale)

# Accuracy по W/D/L
result_accuracy = np.mean(true_result == pred_result)
print("W/D/L accuracy:", result_accuracy)

MSE: 1.4130086040348728
MAE (L1): 0.9222614976919175
R^2: 0.15815700266295818
Custom accuracy (обидва <= 0.5): 0.1174559753804069
W/D/L accuracy: 0.5811249786288254
